In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn import cross_validation
#from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time 
from sklearn import neighbors
from sklearn.utils import shuffle
tf.logging.set_verbosity(tf.logging.ERROR)
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

C:\Users\steja\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
mnist = fetch_mldata("MNIST original")

In [3]:
print(mnist.data.shape)

(70000, 784)


In [4]:
print(np.unique(mnist.target))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


In [5]:
X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X, y = shuffle(X,y)

In [6]:
#X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X_train, y_train = np.float32(X[:15000]), np.float32(y[:15000])
X_test, y_test = np.float32(X[60000:]), np.float32(y[60000:])

# PCA 

In [7]:
pca=PCA(n_components=700)
X_train_pca=pca.fit_transform(X_train,y_train)
X_test_pca=pca.transform(X_test)
print('Variance covered', pca.explained_variance_ratio_.sum())
print('PCA Shape', X_train_pca.shape)

Variance covered 1.0
PCA Shape (15000, 700)


# LDA

In [8]:
lda = LinearDiscriminantAnalysis()
X_train_lda=lda.fit_transform(X_train_pca,y_train)
X_test_lda=lda.transform(X_test_pca)
print('Variance covered', lda.explained_variance_ratio_.sum())
print('LDA Shape', X_train_lda.shape)

C:\Users\steja\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Variance covered 1.0
LDA Shape (15000, 9)


In [9]:
start = int(round(time.time() * 1000))
RF=RandomForestClassifier(random_state=1)
RF.fit(X_train_lda,y_train)
end = int(round(time.time() * 1000))
print("--NN fitting finished in ", (end-start), "ms--------------")
predicted=RF.predict(X_test_lda)
expected=y_test

--NN fitting finished in  1258 ms--------------


In [10]:
print(cross_validation.cross_val_score(RF, X_train_lda,y_train, cv=5))

[ 0.90412783  0.90006662  0.90366667  0.90193462  0.8935247 ]


In [11]:
print("Classification report for kNN classifier %s:\n%s\n"
     % (RF, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Classification report for kNN classifier RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False):
             precision    recall  f1-score   support

        0.0       0.94      0.95      0.95       984
        1.0       0.92      0.94      0.93      1107
        2.0       0.83      0.87      0.85       959
        3.0       0.83      0.85      0.84      1015
        4.0       0.85      0.89      0.87       958
        5.0       0.81      0.81      0.81       892
        6.0       0.93      0.90      0.92      1025
        7.0       0.88      0.88      0.88      1074
        8.0       0.84      0.75      0.79      1009
        9.0       0.8

In [12]:
print("Accuracy is:",round(metrics.accuracy_score(expected,predicted)*100,2))

Accuracy is: 86.91


In [13]:
print("Test error is:",100-round(metrics.accuracy_score(expected,predicted)*100,2))

Test error is: 13.09


In [14]:
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Confusion matrix:
[[ 939    0   14    4    3    4   11    1    7    1]
 [   1 1042   10    7    4    7    0    9   25    2]
 [   9   15  838   20   10    8   16   10   25    8]
 [   4    8   51  866    3   30    2   19   22   10]
 [   4    1   11    2  855    4   23    7    6   45]
 [  15    4   12   64    5  724   12    9   38    9]
 [  14    5   18    2   12   28  926    6   12    2]
 [   3   10   23   12   23    6    0  941    2   54]
 [   7   43   16   43   24   81    7    6  760   22]
 [   5    4   14   18   61    5    1   59   10  800]]
